In [ ]:
from wave_1d_fd_pml import propagators, test_wave_1d_fd_pml, find_profile
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [ ]:
profile_len = 10
x, profile = find_profile_linear(profile_len, intercept_bounds=[5000, 5000], slope_bounds=[-500, 500], vs=[1500], maxiter=500)

In [ ]:
model = test_wave_1d_fd_pml.model_one(400, v0=5000, v1=1500, freq=15)
plt.plot(profile)
v = propagators.Pml(model['model'], model['dx'], model['dt'], len(profile), profile=profile)
y = v.steps(model['nsteps'], model['sources'], model['sx'])
print(np.sum(np.abs(v.current_wavefield)))

plt.figure()
y[:, v.total_pad] = y[:, -v.total_pad] = np.nan
plt.imshow(y[:,:], aspect='auto')
